#  Standard Star Tutorial
Create a simulated observation of a standard A0 star

In [ ]:
from pyechelle.simulator import Simulator
from pyechelle.telescope import Telescope
from pyechelle.sources import Phoenix, Etalon, Constant, CSV
from pyechelle.spectrograph import ZEMAX
from pyechelle.efficiency import TabulatedEfficiency

import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
import astropy.units as u
import astropy.constants as c

import scipy.io
from scipy.ndimage import gaussian_filter
import scipy.integrate as integrate

from psisim import instrument,observation,spectrum,universe,plots,signal,telescope

### Setup paths and file names
Change the output path.

In [ ]:
output_path = '/home/aidan/data/hispec_simulations/'

sci_fiber_model = 'HISPEC_HK_SCI'
lfc_fiber_model = 'HISPEC_HK_LFC'
sky_fiber_model = 'HISPEC_HK_SKY_BKGD'
dark_fiber_model = 'HISPEC_HK_DARK'

### General observing parameters

In [ ]:
t_exp = 30.0 # s
tele = Telescope(9.96,0) # effective diameter of 9.96 meters, 0 central obstructions
star_J_mag = 10.0
planet_K_mag = 20.0

#### Initialize PSISIM

In [ ]:
path = '/scr3/dmawet/ETC/'
filters = spectrum.load_filters()

# telescope
keck = telescope.Keck(path=path)
keck.airmass=1.0
keck.water_vapor=1.6
keck.seeing = keck.median_seeing

# instrument
hispec = instrument.hispec(telescope=keck)

### LFC Fiber

Read in an lfc profile and use it to normalize our own frequency pattern. The resolution of the lfc spectrum is not high enough for use by itself.

In [ ]:
lfc_hk_path = 'lfc/20220513_1.5-3.4um/disp2_2.145_disp3_-0.05_ptamp_4.5A_1.5-2.7um.mat'
lfc_hk = scipy.io.loadmat(lfc_hk_path) 

lfc_hk_power = np.power(10, lfc_hk['OSAPower'] / 10) * 1e4 * u.erg/u.s
lfc_hk_wvs = (lfc_hk['OSAWavelength'] * u.nm).to(u.micron).flatten()
lfc_hk_freq = (c.c / lfc_hk_wvs).to(u.Hz)

# get the low pass envelope of the power
lfc_envelope = gaussian_filter(lfc_hk_power, 30).flatten()
lfc_envelope[0], lfc_envelope[-1] = 0.0, 0.0 # setting the ends to zero so it isn't interpolated beyond the range

Here we create a frequency grid spectrum to mock the LFC

In [ ]:
# constructing frequency grid and converting it to wavelength space
freq_spacing = 10e9 * u.Hz # 10 GHz spacing - not sure what the real value will be
start_freq = (c.c /  0.9 / u.um ).to(u.Hz)
end_freq = (c.c /  2.5 / u.um ).to(u.Hz)
freq_array = np.arange(end_freq.value, start_freq.value, freq_spacing.value) * u.Hz
wave_array = (c.c / freq_array).to(u.um)

# convert power in the lfc envelope to a number of photons at each wavelength 
lfc_photons = (np.interp(wave_array, lfc_hk_wvs.flatten(), lfc_envelope.flatten()) / (c.h * freq_array).to(u.erg))
lfc_photons = lfc_photons / 1e11 # the number of photons is currently scaled down arbitrarily

# save to csv 
lfc_spec = np.transpose([wave_array.value, lfc_photons.value])
np.savetxt(output_path+'lfc_spec_texp'+str(t_exp)+'.csv', lfc_spec, delimiter=',')

Get the throughput for the LFC fiber and spectrograph, which will be used by Pyechelle.

In [ ]:
th_spec = hispec.get_spec_throughput(wave_array)
th_wvs = hispec.th_data[0] * u.micron
th_fiber = np.interp(wave_array, th_wvs, np.prod(hispec.th_data[31:38], axis=0))

Run the simulation for the LFC fiber. Note that some orders may saturate.

In [ ]:
lfc_sim = Simulator(ZEMAX(lfc_fiber_model))
lfc_sim.set_ccd(1) # always leave as 1
lfc_sim.set_fibers(1) # leave as 1 since the fibers are in separate models

lfc_source = CSV(filepath=output_path+'lfc_spec_texp'+str(t_exp)+'.csv',list_like=True,
                 flux_in_photons=True, wavelength_unit='micron')

lfc_sim.set_sources(lfc_source) 
lfc_sim.set_efficiency(TabulatedEfficiency(1, wave_array, th_spec*th_fiber))

lfc_sim.set_exposure_time(t_exp) # s
lfc_sim.set_output(output_path+lfc_fiber_model+'_exp'+str(t_exp)+'.fits', overwrite=True)
lfc_sim.set_telescope(tele) 
lfc_sim.set_orders([*range(59, 98)]) # a list of all expected orders 59 to 97

lfc_sim.max_cpu = 1 # set number of cpu to use
lfc_sim.run()

### Science fiber

Get an A0 phoenix spectrum from PSISIM. We will also collect a sky spectrum and thermal background.
This is essentially a copy of the steps from the PSISIM HISPEC Tutorial notebook, except for an on-axis source. Issues - thermal flux may not be correct, and dark noise is being included twice

In [ ]:
#First set the host properties for a Phoenix model. 
stellar_properties = {"StarLogg":4.00*u.dex(u.cm/ u.s**2),"StarTeff":10000*u.K,"StarZ":'-0.0',"StarAlpha":"0.0",
                   "StarRadialVelocity":100*u.km/u.s,
                    "StarVsini":10*u.km/u.s,
                    "StarLimbDarkening":0.8}

# Hispec doesn't care about the spectral type, but we need to include the paramter
stellar_properties['StarSpT'] = None

# The angular separation is also required, but not actually used when speckle noise is not included
stellar_properties['AngSep'] = 0.0 *u.mas

obj_properties = {"StarLogg":3.25*u.dex(u.cm/ u.s**2),"StarTeff":700*u.K,"StarRadialVelocity":20*u.km/u.s,"StarVsini":10*u.km/u.s,
                  "StarLimbDarkening":0.9}


all_wavelengths = []
full_stellar_spectrum = []
full_thermal_spec = []

for hispec_filter in hispec.filters:
    
    #Setup the instrument
    hispec.set_current_filter(hispec_filter)
    wavelengths = hispec.get_wavelength_range()
    hispec.set_observing_mode(t_exp,1,hispec_filter, wavelengths, mode='on-axis') 
    
    
    stellar_user_params = (path,'TwoMASS-J',star_J_mag,filters,hispec.current_filter)
    
    #Get the host star magnitude in the AO filter
    stellar_properties["StarAOmag"] = spectrum.get_model_ABmags(stellar_properties,[hispec.ao_filter], model='Phoenix',
                                                             verbose=False,user_params = stellar_user_params)
    hispec.ao_mag = stellar_properties["StarAOmag"]
    
    stellar_spectrum = spectrum.get_stellar_spectrum(stellar_properties,wavelengths,hispec.current_R,
                                                  model="Phoenix",user_params=stellar_user_params,
                                                  doppler_shift=True,broaden=True,delta_wv=hispec.current_dwvs)
    
    obj_user_params = (path,'TwoMASS-K',planet_K_mag,filters,hispec.current_filter)

    obj_spectrum = spectrum.get_stellar_spectrum(obj_properties,wavelengths,hispec.current_R,model="Sonora",
                                             user_params=obj_user_params,doppler_shift=True,broaden=True,
                                             delta_wv=hispec.current_dwvs)
    
    obj_spectrum.spectrum /= stellar_spectrum.spectrum
        
    obj_spec,total_noise,stellar_spec,thermal_spec,noise_components= observation.simulate_observation(keck,hispec,
                                                                                      stellar_properties,
                                                                                      obj_spectrum.spectrum.value,wavelengths,1e5,
                                                                                      inject_noise=False,verbose=True,
                                                                                      post_processing_gain = np.inf,
                                                                                      return_noise_components=True,
                                                                                      stellar_spec=stellar_spectrum.spectrum,
                                                                                      apply_lsf=False,
                                                                                      integrate_delta_wv=False,
                                                                                      plot=False,
                                                                                      sky_on=True)
    
    
    all_wavelengths.append(wavelengths)
    full_stellar_spectrum.append(stellar_spec)
    full_thermal_spec.append(thermal_spec)
    
all_wavelengths = np.hstack(all_wavelengths).value*wavelengths.unit
full_stellar_spectrum = np.hstack(full_stellar_spectrum).value*stellar_spec.unit
full_thermal_spec = np.hstack(full_thermal_spec).value*thermal_spec.unit

In [ ]:
# science + sky / thermal
sci_spec = np.transpose([all_wavelengths.value, full_stellar_spectrum.value+full_thermal_spec.value])
np.savetxt(output_path+'sci_spec_exp'+str(t_exp)+'_Jmag'+str(star_J_mag)+'.csv', sci_spec, delimiter=',')

# sky / thermal only
sky_spec = np.transpose([all_wavelengths.value, full_thermal_spec.value])
np.savetxt(output_path+'sky_spec_exp'+str(t_exp)+'.csv', sky_spec, delimiter=',')

Simulate

In [ ]:
sci_sim = Simulator(ZEMAX(sci_fiber_model))
sci_sim.set_ccd(1) # always leave as 1
sci_sim.set_fibers(1) # leave as 1 since the fibers are in separate models

sci_source = CSV(filepath=output_path+'sci_spec_exp'+str(t_exp)+'_Jmag'+str(star_J_mag)+'.csv',list_like=False,
                 flux_in_photons=True, wavelength_unit='micron')

sci_sim.set_sources(sci_source) 

sci_sim.set_exposure_time(t_exp) # s
sci_sim.set_output(output_path+sci_fiber_model+'_exp'+str(t_exp)+'_Jmag'+str(star_J_mag)+'.fits', overwrite=True)
sci_sim.set_telescope(tele) 

# setting orders that do not have any data will throw an error
sci_sim.set_orders([62,63,64,65,66,67,68,69,70,71,72,72,74,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,96,96,97]) 
# the efficiency has already been applied to the number of photons in the csv
sci_sim.set_efficiency(TabulatedEfficiency(1, [0.9,2.5], [1,1]))

sci_sim.max_cpu = 1 # set number of cpu to use
sci_sim.run()

### Sky fiber


In [ ]:
if full_thermal_spec[0].value > 1.0:
    # only do the sky simulation if there are actually photons to simulate
    sky_sim = Simulator(ZEMAX(sky_fiber_model))
    sky_sim.set_ccd(1) # always leave as 1
    sky_sim.set_fibers(1) # leave as 1 since the fibers are in separate models

    sky_source = CSV(filepath=output_path+'sky_spec_exp'+str(t_exp)+'.csv',list_like=False,
                     flux_in_photons=True, wavelength_unit='micron')

    sky_sim.set_sources(sky_source) 
    sky_sim.set_exposure_time(t_exp) # s
    sky_sim.set_output(output_path+sky_fiber_model+'_exp'+str(t_exp)+'.fits', overwrite=True)
    sky_sim.set_telescope(tele) 
    sky_sim.set_orders([62,63,64,65,66,67,68,69,70,71,72,72,74,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,96,96,97]) 
    # the efficiency has already been applied to the number of photons in the csv
    sky_sim.set_efficiency(TabulatedEfficiency(1, [0.9,2.5], [1,1]))

    sky_sim.max_cpu = 1 # set number of cpu to use
    sky_sim.run()

### Create noise background

In [ ]:
import nghxrg as ng
# Instantiate a new object on account of the different array dimensions.
# Recall that the H4RG has 4096x4096 pixels. Run using
# 32 outputs. Also set the new row overhead to 8 pixels (a power of 2)
# which simplifies working with the data in Fourier space.
ng_h4rg = ng.HXRGNoise(naxis1=4096, naxis2=4096, n_out=32, nroh=8, verbose=False)

# Make a noise file.
rd_noise=4   # White read noise per integration
pedestal=4   # DC pedestal drift rms
c_pink=3     # Correlated pink noise
u_pink=1     # Uncorrelated pink noise
c_ACN=1      # Alternating column noise

# Use the same parameters as in Ex. 2.1
acn=.5        # Correlated ACN
pca0_amp=.2   # Amplitude of PCA zero "picture frame" noise

my_hdu = ng_h4rg.mknoise(output_path+'/h4rg_noise.fits', rd_noise=rd_noise, pedestal=pedestal, c_pink=c_pink,
            u_pink=u_pink, acn=acn)

Dark current

In [ ]:
# Setup
i_dark = 0.005# e-/s/pix - mean current of NIRSpec detectors

# Open the result of Ex. 2.1.1
hdulist = fits.open(output_path+'h4rg_noise.fits')

# Add Poisson noise to the data
d = hdulist[0].data + np.random.poisson(i_dark*t_exp, np.shape(hdulist[0].data))

# Write result
hduout = fits.PrimaryHDU(d)
hduout.writeto(output_path+'h4rg_noise_w_dark_current.fits', clobber=True)

# Clean up
hdulist.close()

In [ ]:
hdu_noise = fits.open(output_path+'h4rg_noise_w_dark_current.fits')
hdu_lfc = fits.open(output_path+lfc_fiber_model+'_exp'+str(t_exp)+'.fits')
hdu_sci = fits.open(output_path+sci_fiber_model+'_exp'+str(t_exp)+'_Jmag'+str(star_J_mag)+'.fits')
hdu_sky = fits.open(output_path+sky_fiber_model+'_exp'+str(t_exp)+'.fits')


d = hdu_noise[0].data + hdu_lfc[0].data + hdu_sci[0].data + hdu_sky[0].data

# Write result
hduout = fits.PrimaryHDU(d)
hduout.writeto(output_path+'HISPECsim_out_'+str(t_exp)+'s.fits', overwrite=True)

# Clean up
hdulist.close()